In [1]:
import pandas as pd
import my_utils as mu
import json

In [2]:
missing_item = mu.getMatchDataAndTimelineByMatchId('KR_6486524488', '')

In [3]:
missing_item = {'matches': missing_item[0], 'timeline': missing_item[1]}

In [6]:
mu.getEventList(missing_item['timeline'])

['CHAMPION_SPECIAL_KILL',
 'PAUSE_END',
 'SKILL_LEVEL_UP',
 'WARD_KILL',
 'ELITE_MONSTER_KILL',
 'OBJECTIVE_BOUNTY_PRESTART',
 'GAME_END',
 'ITEM_DESTROYED',
 'TURRET_PLATE_DESTROYED',
 'CHAMPION_KILL',
 'ITEM_PURCHASED',
 'BUILDING_KILL',
 'WARD_PLACED',
 'LEVEL_UP',
 'ITEM_UNDO',
 'ITEM_SOLD']

In [7]:
destroyed_items = [a['itemId'] for a in mu.eventExtractor(missing_item, 'ITEM_DESTROYED')]

### getHoldingItems() 작성중...

#### 'ITEM_DESTROYED' 반영 필요

In [15]:
tmp = 10
0 <= tmp <= 20

True

In [17]:
def getHoldingItems(raw_data_series: pd.Series, part_number: int, start_time: int=0, end_time: int=60000):
    
	# 아이템 구매, 판매, 되돌리기 이벤트를 종합
	errs = \
		[{'timestamp':purchased['timestamp'], 'type':purchased['type'], 'itemId':purchased['itemId']}\
		for purchased in list(filter(lambda x: start_time <= x['timestamp'] <= end_time, mu.eventExtractor(raw_data_series, 'ITEM_PURCHASED', part_number)))] + \
		[{'timestamp':sold['timestamp'], 'type':sold['type'], 'itemId':sold['itemId']}\
		for sold in list(filter(lambda x: start_time <= x['timestamp'] <= end_time, mu.eventExtractor(raw_data_series, 'ITEM_SOLD', part_number)))] + \
		[{'timestamp':undo['timestamp'], 'type':undo['type'], 'beforeId':undo['beforeId'], 'afterId':undo['afterId']}\
		for undo in list(filter(lambda x: start_time <= x['timestamp'] <= end_time, mu.eventExtractor(raw_data_series, 'ITEM_UNDO', part_number)))]

	# timestamp로 이벤트를 시간 순 정렬
	errs.sort(key=lambda x: x['timestamp'])

	# 되돌리기 이벤트와 되돌려진 이벤트 제거
	evt_list = []
	for evt in errs:
		try:
			if evt['type'] == 'ITEM_PURCHASED': evt_list.append(evt)
			elif evt['type'] == 'ITEM_SOLD': evt_list.append(evt)
			else: evt_list.pop() # UNDO 이벤트 발생시 최근 이벤트 제거
		except IndexError as e:
			pass ### UNDO가 가장 먼저 나오는 케이스에 대한 예외 처리 및 로깅 기능 적용 예정
		except Exception as e:
			pass ### 이외의 예측하지 못한 예외에 대한 로깅 기능 적용 예정

	# 정제된 이벤트 리스트로 시작 아이템을 추적하여 리턴
	start_items = []
	for evt in evt_list:
		try:
			if evt['type'] == 'ITEM_PURCHASED': start_items.append(evt['itemId'])
			else: start_items.remove(evt['itemId'])
		except ValueError as e:
			pass ### 구매하지 않은 아이템이 판매되는 경우에 대한 예외 처리 및 로깅 기능 적용 예정
		except Exception as e:
			pass ### 이외의 예측하지 못한 예외에 대한 로깅 기능 적용 예정
	return start_items

In [10]:
part = 2
until = 600000
errs = \
    [{'timestamp':purchased['timestamp'], 'type':purchased['type'], 'itemId':purchased['itemId']}\
    for purchased in list(filter(lambda x: x['timestamp'] <= until, mu.eventExtractor(missing_item, 'ITEM_PURCHASED', part)))] + \
    [{'timestamp':sold['timestamp'], 'type':sold['type'], 'itemId':sold['itemId']}\
    for sold in list(filter(lambda x: x['timestamp'] <= until, mu.eventExtractor(missing_item, 'ITEM_SOLD', part)))] + \
    [{'timestamp':undo['timestamp'], 'type':undo['type'], 'beforeId':undo['beforeId'], 'afterId':undo['afterId']}\
    for undo in list(filter(lambda x: x['timestamp'] <= until, mu.eventExtractor(missing_item, 'ITEM_UNDO', part)))] + \
    [{'timestamp':destroyed['timestamp'], 'type':destroyed['type'], 'itemId':destroyed['itemId']}\
    for destroyed in list(filter(lambda x: x['timestamp'] <= until, mu.eventExtractor(missing_item, 'ITEM_DESTROYED', part)))]
errs.sort(key=lambda x: x['timestamp'])

In [11]:
errs

[{'timestamp': 14054, 'type': 'ITEM_PURCHASED', 'itemId': 1103},
 {'timestamp': 14492, 'type': 'ITEM_PURCHASED', 'itemId': 3340},
 {'timestamp': 14726, 'type': 'ITEM_PURCHASED', 'itemId': 2003},
 {'timestamp': 54583, 'type': 'ITEM_PURCHASED', 'itemId': 3364},
 {'timestamp': 54583, 'type': 'ITEM_DESTROYED', 'itemId': 3340},
 {'timestamp': 176212, 'type': 'ITEM_DESTROYED', 'itemId': 2003},
 {'timestamp': 243463, 'type': 'ITEM_PURCHASED', 'itemId': 3158},
 {'timestamp': 249741, 'type': 'ITEM_PURCHASED', 'itemId': 2055},
 {'timestamp': 249774, 'type': 'ITEM_PURCHASED', 'itemId': 2055},
 {'timestamp': 318166, 'type': 'ITEM_PURCHASED', 'itemId': 1028},
 {'timestamp': 370366, 'type': 'ITEM_DESTROYED', 'itemId': 2055},
 {'timestamp': 401494, 'type': 'ITEM_DESTROYED', 'itemId': 2055},
 {'timestamp': 414627, 'type': 'ITEM_PURCHASED', 'itemId': 3067},
 {'timestamp': 414627, 'type': 'ITEM_DESTROYED', 'itemId': 1028},
 {'timestamp': 422474, 'type': 'ITEM_PURCHASED', 'itemId': 2055},
 {'timestamp': 

In [19]:
getHoldingItems(missing_item, 1)

[1054, 3340, 2031]

### eventExtractor() 수정 사항

In [ ]:
public = [
    'OBJECTIVE_BOUNTY_PRESTART',
    'GAME_END',
    'PAUSE_END'
]
private = [
    ('CHAMPION_KILL', 'killerId', int),
    ('BUILDING_KILL', 'killerId', int),
    ('TURRET_PLATE_DESTROYED', 'killerId', int),
    ('ELITE_MONSTER_KILL', 'killerId', int),
    ('CHAMPION_SPECIAL_KILL', 'killerId', int),
    ('WARD_KILL', 'killerId', int),
    ('WARD_PLACED', 'creatorId', int)
]
adder = [
    ('CHAMPION_ASSIST', 'assistingParticipantIds', list),
    ('CHAMPION_DEATH', 'victimId', int)
]